In [1]:
from onnx import shape_inference
import onnx_graphsurgeon as gs
import numpy as np
import onnx

In [8]:
@gs.Graph.register()
def add(self, a, b):
    return self.layer(op="Add", inputs=[a, b], outputs=["add_out_gs"])

In [9]:
@gs.Graph.register()
def identity(self, a):
    return self.layer(op="Identity", inputs=[a], outputs=["identy_out_gs"])

In [14]:
@gs.Graph.register()
def identityAdd(self,a,b):
  return self.identity(*self.add(a,b))

In [10]:
@gs.Graph.register()
def sub(self, a, b):
    return self.layer(op="Sub", inputs=[a, b], outputs=["sub_out_gs"])

In [11]:
@gs.Graph.register()
def addAndSub(self,a,b,c,d):
  ab = self.add(a,b)
  cd = self.add(c,d)
  return self.sub(*ab,*cd)

In [19]:
graph = gs.Graph(opset=13)


In [ ]:
input1 = gs.Variable(name="input", shape=(1024, 22), dtype=np.int64)

graph.inputs = [input]
offest = np.ones(shape=(1024, 22), dtype=np.int64)
offset2 = np.ones(shape=(1024, 22), dtype=np.int64)
graph.outputs = graph.addAndSub(input,offest,input,offset2)
for out in graph.outputs:
    out.dtype = np.int64
model = gs.export_onnx(graph)
model = shape_inference.infer_shapes(model)
onnx.save(model, "model2.onnx")


In [20]:
# mock一个模型，由一个add和一个identity组成，因为对于输出为output的节点的冗余改写复杂，在推荐系统也不存在，
# 现在目标是先改写一个输出为非输出节点的add算子
input1 = gs.Variable(name="input", shape=(1024, 22), dtype=np.int64)

graph.inputs = [input1]
offest = np.ones(shape=(1024, 22), dtype=np.int64)
graph.outputs = graph.identityAdd(input1,offest)
for out in graph.outputs:
    out.dtype = np.int64
model = gs.export_onnx(graph)
model = shape_inference.infer_shapes(model)
onnx.save(model, "../test_model/model_identity.onnx")

In [21]:
from onnxruntime import SessionOptions,ExecutionMode
import onnxruntime as ort
sess_opt = SessionOptions()
sess_opt.execution_mode  = ExecutionMode.ORT_PARALLEL
# sess_opt.inter_op_num_threads = 3
session = ort.InferenceSession("../test_model/model_identity.onnx",sess_options = sess_opt,providers=[ 'CPUExecutionProvider'])
for i in range(10):
  random_input1 = np.random.rand(1024,22).astype(np.int64)
  out = session.run([],{"input":random_input1})

In [1]:
import onnx_utils

In [22]:
@gs.Graph.register()
def slice2(self, name, input, starts, ends, axes):
    """
    Add Slice operation to the graph which will operate on the input tensor with the value(s) given.
    :param op: The ONNX operation to perform, i.e. "Add" or "Mul".
    :param input: The tensor to operate on.
    :param name: The name to use for the node.
    :param starts: Value at which Slice starts.
    :param ends: Value at which Slice ends.
    :param axes: Axes on which Slice operation should be performed.
    """

    input_tensor = input if type(input) is gs.Variable else input[0]
    const_start = gs.Constant(
        name="{}_value:0".format(name), values=np.asarray([starts], dtype=np.int64)
    )
    const_end = gs.Constant(
        name="{}_value:1".format(name), values=np.asarray([ends], dtype=np.int64)
    )
    const_axes = gs.Constant(
        name="{}_value:2".format(name), values=np.asarray([axes], dtype=np.int64)
    )
    return self.layer(
        name=name,
        op="Slice",
        inputs=[input_tensor],
        outputs=[name + ":0"],
        attrs={'axes':axes,'starts':starts,'ends':ends}
    )

In [28]:
graph = gs.Graph(opset=13)


In [29]:
input = gs.Variable(name="input", shape=(1024, 22), dtype=np.int64)


In [30]:
graph.inputs = [input]
graph.outputs = graph.slice('reSlice',input,0,12,0)
for out in graph.outputs:
    out.dtype = np.int64
model = gs.export_onnx(graph)
model = shape_inference.infer_shapes(model)
onnx.save(model, "slicemodel.onnx")

In [31]:
graph.nodes

[reSlice (Slice)
 	Inputs: [
 		Variable (input): (shape=(1024, 22), dtype=<class 'numpy.int64'>)
 		Constant (reSlice_value:0): (shape=(1,), dtype=int64)
 		Constant (reSlice_value:1): (shape=(1,), dtype=int64)
 		Constant (reSlice_value:2): (shape=(1,), dtype=int64)
 	]
 	Outputs: [
 		Variable (reSlice:0_0): (shape=None, dtype=<class 'numpy.int64'>)
 	]]

In [32]:
from onnxruntime import SessionOptions,ExecutionMode
import onnxruntime as ort
sess_opt = SessionOptions()
sess_opt.execution_mode  = ExecutionMode.ORT_PARALLEL
# sess_opt.inter_op_num_threads = 3
session = ort.InferenceSession("/home/yssun/onnx/enode/slicemodel.onnx",sess_options = sess_opt,providers=[ 'CPUExecutionProvider'])
for i in range(10):
  random_input1 = np.random.rand(1024,22).astype(np.int64)
  out = session.run([],{"input":random_input1})